In [1]:
# Unsloth 및 의존성 설치 (시간이 조금 걸립니다)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ky3sxqiz/unsloth_2b01210ebfde4d908a7372116b302688
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ky3sxqiz/unsloth_2b01210ebfde4d908a7372116b302688
  Resolved https://github.com/unslothai/unsloth.git to commit 8ea5338154859ed25b50366cb1264ed4d933eae3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached trl-0.24.0-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.24.0-py3-none-any.whl (423 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Attempting uninstall: trl
    Found existing installation: trl 0.24.0
    Uninstalling trl-0.24.0:
      Successfully 

In [2]:
import torch
import os
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
from google.colab import drive
# Google Drive 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 설정
model_name = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit"
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
# 모델 및 토크나이저 로드
print(f" 모델 로드 중... ({model_name})")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# LoRA 어댑터 설정
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🔄 모델 로드 중... (unsloth/Qwen3-0.6B-unsloth-bnb-4bit)
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.12.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
# 공통으로 사용할 시스템 프롬프트
system_prompt = "당신은 6G 이동통신과 AI 분야의 전문가입니다. 질문에 대해 항상 한국어로 명확하고 전문적으로 답변하세요."
question_text = "6G 네트워크에서 AI의 역할은 무엇인가요?"

In [ ]:
# 학습 전(Before) 추론 테스트
print("\n" + "="*50)
print(" [Before] 학습 전 기본 모델 테스트")
print("="*50)

# 추론 모드 전환
FastLanguageModel.for_inference(model)

# 메시지 구성 (System 프롬프트 추가)
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question_text + " (한국어로 설명해줘)"}
]

# 토큰화
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# 답변 생성
_ = model.generate(
    input_ids = inputs,
    streamer = TextStreamer(tokenizer, skip_prompt=True),
    max_new_tokens = 512,
    use_cache = True,
    temperature = 0.7,
)
print("\n" + "="*50 + "\n")


🛑 [Before] 학습 전 기본 모델 테스트
<think>
Okay, the user is asking about the role of AI in 6G networks. I need to explain it clearly in Korean. Let me start by recalling what I know. 6G is the next generation of wireless technology, so it's going to be much faster, more efficient, and have more capabilities. AI is going to play a big role here.

First, I should mention how AI will optimize network performance. For example, using machine learning to predict and adapt to user behavior. That would make the network more responsive. Then, AI might help in resource allocation, like dynamically assigning bandwidth to different users. That would improve overall efficiency.

I need to highlight how AI can enhance security and reliability. AI can detect anomalies and ensure data privacy. Also, AI might support advanced features like quantum computing, which could be integrated into 6G. But wait, the user might not know about quantum computing yet. So, I should mention that.

Another point is how AI can

In [ ]:
# 데이터셋 로드 및 포맷팅
dataset_file = "/content/drive/MyDrive/js_mujoco/learn_LLM/src/6g_ai_dataset_augmented.jsonl"

if not os.path.exists(dataset_file):
    raise FileNotFoundError(f" 데이터 파일을 찾을 수 없습니다: {dataset_file}\n파일을 업로드하거나 경로를 수정해주세요.")

dataset = load_dataset("json", data_files=dataset_file, split="train")

# Qwen 채팅 템플릿 적용
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        messages = [
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": output}
        ]
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        texts.append(text)
    return {"text": texts}

train_dataset = dataset.map(formatting_prompts_func, batched=True)
print(f" 데이터 준비 완료! (샘플): {train_dataset[0]['text'][:100]}...")

✅ 데이터 준비 완료! (샘플): <|im_start|>user
2025 6G White Paper의 주요 주제는 무엇입니까?<|im_end|>
<|im_start|>assistant
<think>

</think...


In [ ]:
# 학습 (Training)
print(" 학습을 시작합니다...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # 데이터 양에 따라 조절 (60은 테스트용으로 적절)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # 완디비(wandb) 끄기
    ),
)

trainer_stats = trainer.train()
print(" 학습 완료!")

🚀 학습을 시작합니다...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 36 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 10,092,544 of 606,142,464 (1.67% trained)


Step,Training Loss
10,5.977800
20,6.085300
30,6.056000
40,6.143400
50,6.087500
60,6.025000


🎉 학습 완료!


In [ ]:
# 추론 테스트 (Inference)
print("\n" + "="*50)
print(" [After] 학습 후 파인튜닝 모델 테스트 ")
print("="*50)

# 추론 모드 전환 (혹시 학습 모드였다면 다시 변경)
FastLanguageModel.for_inference(model)

# 메시지 구성
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question_text + " (한국어로 설명해줘)"}
]

# 토큰화
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

# 답변 생성
_ = model.generate(
    input_ids = inputs,
    streamer = TextStreamer(tokenizer, skip_prompt=True),
    max_new_tokens = 512,
    use_cache = True,
    temperature = 0.7,
)
print("\n" + "="*50)


✅ [After] 학습 후 파인튜닝 모델 테스트 
<think>
Okay, the user is asking about the role of AI in 6G networks. I need to explain it in Korean. First, I should recall what 6G is. It's a future network, right? So the question is about how AI contributes to that.

I should start by mentioning AI's main functions. Maybe things like data processing, optimization, and security. Then, I can talk about specific technologies like machine learning models, quantum computing, and edge computing. Also, AI helps in enhancing the network's performance, enabling better services, and improving security. Need to make sure it's clear and flows well in Korean. Check that the explanation covers the key points without being too technical, keeping it simple. Make sure the answer is concise and addresses the user's query.
</think>

6G 네트워크에서 AI는 기존의 통신 기술보다 더욱 나은 서비스를 제공하고, 신속한 대응과 효율성을 높이는 역할을 담당합니다. AI 기반의 인공지능 모델은 네트워크 구축, 운영, 보호, 서비스 개선에 중요한 역할을 하며, 다음의 주요 기능을 통해 6G를 성공적으로 구현할 수 있습니다:

1. **데이터 처리 및 분석**: AI는海量 데이터를 

In [ ]:
# 먼저 로컬(Colab)에 모델 저장
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
print(" 로컬 저장 완료: lora_model")

# 구글 드라이브로 복사
import shutil

drive_path = "/content/drive/MyDrive/my_finetuned_qwen"

if os.path.exists("lora_model"):
    shutil.copytree("lora_model", drive_path, dirs_exist_ok=True)
    print(f" 구글 드라이브 백업 완료: {drive_path}")
else:
    print(" 여전히 폴더가 없습니다. 위 저장 코드가 제대로 실행되었는지 확인하세요.")

✅ 로컬 저장 완료: lora_model
🚀 구글 드라이브 백업 완료: /content/drive/MyDrive/my_finetuned_qwen


런타임 세션 다시 시작 후 테스트

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer

# 1. 저장된 경로 설정
save_dir = "/content/drive/MyDrive/my_finetuned_qwen"

# 2. 모델 로드
print(f"📂 모델을 불러오는 중입니다: {save_dir}")
loaded_model, loaded_tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_dir,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# 3. 추론 모드 설정
FastLanguageModel.for_inference(loaded_model)

# 4. 질문 설정 (한국어 강제 프롬프트 포함)
messages = [
    # 시스템 프롬프트: 한국어 전문가 페르소나 부여
    {"role": "system", "content": "당신은 6G 이동통신과 AI 분야의 전문가입니다. 질문에 대해 항상 한국어로 명확하고 전문적으로 답변하세요."},

    # 사용자 질문
    {"role": "user", "content": "6G 네트워크에서 AI의 역할은 무엇인가요? (한국어로 설명해줘)"}
]

# 5. 토큰화 및 실행
inputs = loaded_tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

print("\n🤖 답변 생성 중...\n")

# 6. 답변 생성
_ = loaded_model.generate(
    input_ids=inputs,
    streamer=TextStreamer(loaded_tokenizer, skip_prompt=True),
    max_new_tokens=512,
    use_cache=True,
    temperature=0.7,
)

📂 모델을 불러오는 중입니다: /content/drive/MyDrive/my_finetuned_qwen
==((====))==  Unsloth 2025.12.9: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



🤖 답변 생성 중...

<think>
Okay, the user is asking about the role of AI in 6G networks. Let me start by recalling what I know. 6G is a future network concept, so it's going to be more advanced than current 5G. AI is a key component here, so I need to explain how AI can be used in 6G.

First, I should mention the integration of AI with 6G. AI will play a critical role in processing and analyzing data, which is essential for 6G's capabilities. Then, maybe talk about applications like autonomous vehicles, smart cities, etc. It's important to highlight how AI will improve efficiency and user experience.

Wait, I need to make sure the explanation is clear and in Korean. Also, check if there's any technical details I should include, like machine learning models or neural networks. Make sure the answer is concise but covers all aspects the user asked for.
</think>

6G 네트워크에서 AI는 기술적 기능과 협력하여 우주형 통신 기술을 구현하고, 이론적 기대치를 현실화하기 위한 핵심 역할을 하며, 다음과 같은 전역적 기능을 통해 이동통신과 인공지능을 결합하여 차이점이 빛납니다.

1. **다양한 데이터